In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
def trans_conv(X, K):
    '''
    X: tensor([[0., 1.],
            [2., 3.]])
    K: tensor([[0., 1.],
            [2., 3.]])
    '''
    h, w = K.shape
    '''
    h, w: 2 2
    '''
    Y = torch.zeros(
        (X.shape[0] + h - 1, X.shape[1] + w - 1)
    )

    '''
    X.shape[0] + h - 1: 3
    X.shape[1] + 2 - 1: 3
    Y: tensor([[0., 0., 0.],
            [0., 0., 0.],
            [0., 0., 0.]])
    '''
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            Y[i: i + h, j: j + w] += X[i, j] * K
            '''
            i: 0
            j: 0
            Y: tensor([[0., 0., 0.],
                    [0., 0., 0.],
                    [0., 0., 0.]])
                    
            i: 0
            j: 1
            Y: tensor([[0., 0., 1.],
                    [0., 2., 3.],
                    [0., 0., 0.]])
                    
            i: 1
            j: 0
            Y: tensor([[0., 0., 1.],
                    [0., 4., 3.],
                    [4., 6., 0.]])
                    
            i: 1
            j: 1
            Y: tensor([[ 0.,  0.,  1.],
                    [ 0.,  4.,  6.],
                    [ 4., 12.,  9.]])
            '''
        
    return Y

In [3]:
X = torch.tensor(
    [
        [0.0, 1.0], [2.0, 3.0]
    ]
)
K = torch.tensor(
    [
        [0.0, 1.0], [2.0, 3.0]
    ]
)
trans_conv(X, K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [4]:
X, K = X.reshape(1, 1, 2, 2), K.reshape(1, 1, 2, 2)
'''
`nn.ConvTranspose2d` 期望输入的数据形状为
`(batch_size, channels, height, width)`。
'''
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

填充、步幅、多通道

In [5]:
tconv = nn.ConvTranspose2d(1, 1, kernel_size=2, padding=1, bias=False)
tconv.weight.data = K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

In [6]:
tconv = nn.ConvTranspose2d(
    1, 1, kernel_size=2, stride=2, bias=False
)
tconv.weight.data = K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [7]:
X = torch.rand(size=(1, 10, 16, 16))
conv = nn.Conv2d(
    10, 20, kernel_size=5, padding=2, stride=3
)
tconv = nn.ConvTranspose2d(
    20, 10, kernel_size=5, padding=2, stride=3
)
'''
conv(X).shape: torch.Size([1, 20, 6, 6])

tconv(conv(X)).shape: torch.Size([1, 10, 16, 16])
'''
tconv(
    conv(X)
).shape == X.shape

True

与矩阵变化的联系

In [8]:
X = torch.arange(9.0).reshape(3, 3)
'''
X： tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
'''
K = torch.tensor(
    [
        [1.0, 2.0], [3.0, 4.0]
    ]
)
Y = d2l.corr2d(X, K)
'''
Y: tensor([[27., 37.],
        [57., 67.]])
'''

'\nY: tensor([[27., 37.],\n        [57., 67.]])\n'

In [9]:
def kernel2matrix(K):
    '''
    K: tensor([[1., 2.],
            [3., 4.]])
    '''
    k_, W = torch.zeros(5), torch.zeros((4, 9))
    '''
    k_: tensor([0., 0., 0., 0., 0.])
    W: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0.]])
    '''
    k_[:2], k_[3:5] = K[0, :], K[1, :]
    '''
    k_: tensor([1., 2., 0., 3., 4.])
    '''
    W[0, :5], W[1, 1:6], W[2, 3:8], W[3, 4:] = k_, k_, k_, k_
    print("W:",W)
    '''
    W: tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
               [0., 1., 2., 0., 3., 4., 0., 0., 0.],
               [0., 0., 0., 1., 2., 0., 3., 4., 0.],
               [0., 0., 0., 0., 1., 2., 0., 3., 4.]])
    '''
    return W

In [10]:
W = kernel2matrix(K)
W

W: tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])


tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [11]:
'''
X: tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])
X.reshape(-1): tensor([0., 1., 2., 3., 4., 5., 6., 7., 8.])
torch.matmul(W, X.reshape(-1)): tensor([27., 37., 57., 67.])
torch.matmul(W, X.reshape(-1)).reshape(2,2): tensor([[27., 37.],
        [57., 67.]])
Y: tensor([[27., 37.],
        [57., 67.]])

'''
Y == torch.matmul(W, X.reshape(-1)).reshape(2,2)

tensor([[True, True],
        [True, True]])

In [12]:
'''
Y: tensor([[27., 37.],
        [57., 67.]])
K: tensor([[1., 2.],
        [3., 4.]])
'''
Z = trans_conv(Y, K)
'''

Z: tensor([[ 27.,  91.,  74.],
        [138., 400., 282.],
        [171., 429., 268.]])
W: tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])
W.T: tensor([[1., 0., 0., 0.],
        [2., 1., 0., 0.],
        [0., 2., 0., 0.],
        [3., 0., 1., 0.],
        [4., 3., 2., 1.],
        [0., 4., 0., 2.],
        [0., 0., 3., 0.],
        [0., 0., 4., 3.],
        [0., 0., 0., 4.]])
Y.reshape(-1): tensor([27., 37., 57., 67.])
torch.matmul(W.T, Y.reshape(-1)): tensor([ 27.,  91.,  74., 138., 400., 282., 171., 429., 268.])
Z: tensor([[ 27.,  91.,  74.],
        [138., 400., 282.],
        [171., 429., 268.]])
'''
Z == torch.matmul(W.T, Y.reshape(-1)).reshape(3, 3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])